In [1]:
%matplotlib inline
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import stats
import powerlaw 
import seaborn as sns

In [2]:
train=pd.read_csv('train.csv')
#test=pd.read_csv('test.csv')
valid=pd.read_csv('validation.csv')
len(valid)

299749

In [4]:
import sqlite3
from math import sqrt
from sklearn.feature_extraction import DictVectorizer as DV
#from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier as RF
#from sklearn.metrics import roc_auc_score as AUC
import statsmodels.api as sm


In [5]:
train["area"]=train["slotheight"]*train["slotwidth"]
valid["area"]=valid["slotheight"]*valid["slotwidth"]

In [6]:
#train_cols = data1.columns[1:]
from sklearn import preprocessing
import statsmodels.api as sm
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
label_encoder = preprocessing.LabelEncoder()
train['hour']=label_encoder.fit_transform(train['hour'])
train['weekday']=label_encoder.fit_transform(train['weekday'])
train['advertiser']=label_encoder.fit_transform(train['advertiser'])
train['area']=label_encoder.fit_transform(train['area'])
train['adexchange']=label_encoder.fit_transform(train['adexchange'])
train['slotprice']=label_encoder.fit_transform(train['slotprice'])
train['slotformat']=label_encoder.fit_transform(train['slotformat'])
train['slotvisibility']=label_encoder.fit_transform(train['slotvisibility'])

features =['weekday', 'hour','advertiser','area','adexchange','slotprice','slotformat','slotvisibility']

valid['hour']=label_encoder.fit_transform(valid['hour'])
valid['weekday']=label_encoder.fit_transform(valid['weekday'])
valid['advertiser']=label_encoder.fit_transform(valid['advertiser'])
valid['area']=label_encoder.fit_transform(valid['area'])
valid['adexchange']=label_encoder.fit_transform(valid['adexchange'])
valid['slotprice']=label_encoder.fit_transform(valid['slotprice'])
valid['slotformat']=label_encoder.fit_transform(valid['slotformat'])
valid['slotvisibility']=label_encoder.fit_transform(valid['slotvisibility'])

In [10]:
from sklearn import ensemble
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

In [114]:
x_train=train[features]
y_train=train['click']
x_test=valid[features]

In [135]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
#rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
rf = RandomForestClassifier(n_estimators=20, max_depth=7, random_state=0)
clf = BaggingClassifier(rf, n_estimators=40, max_samples=0.1, random_state=25)
clf.fit(x_train, y_train)
y_pred2 = clf.predict_proba(x_test)
y_pred2

In [162]:

params = {'n_estimators': 250, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.05, 'loss': 'deviance'}
clf = ensemble.GradientBoostingClassifier(**params)

clf.fit(x_train, y_train)

y_pred1 = clf.predict_proba(x_test)
y_pred1

array([[  9.98989495e-01,   1.01050509e-03],
       [  9.93105198e-01,   6.89480171e-03],
       [  9.98859870e-01,   1.14012961e-03],
       ..., 
       [  9.99476845e-01,   5.23154576e-04],
       [  9.99757274e-01,   2.42725763e-04],
       [  9.98756041e-01,   1.24395867e-03]])

In [163]:
pre_1=[]
for a in y_pred1:
    pre_1.append(a[1])
len(pre_1)

299749

In [164]:
pre_2=[]
for b in y_pred2:
    pre_2.append(b[1])
len(pre_2)

299749

In [165]:
y_pre=valid['click']

In [286]:
pre_3=[]
for i in range(0,299749):
    pre=0.9*pre_1[i]+0.1*pre_2[i]
    pre_3.append(pre)

In [287]:
from sklearn.metrics import mean_squared_error, make_scorer
mse = mean_squared_error(y_pre, pre_3, sample_weight=None, multioutput='uniform_average')
np.sqrt(mse)

0.027434422574643239

In [288]:
import numpy as np
from sklearn.metrics import roc_auc_score
auc=roc_auc_score(y_pre, pre_3)
auc

0.69233133780055423

In [289]:
avg_CTR=sum(train['click'])/len(train)
avg_CTR

0.0007539649884458758

In [370]:
#base_bid=sum(train['bidprice'])/len(train)
#from statistics import mode
base_bid=70 #average bid, 150

In [371]:
L1=valid['bidprice'].values/1000
L2=valid['payprice'].values/1000

In [372]:
L=[]
for i in range(0,299749):
    p_bid=base_bid*(pre_3[i])/avg_CTR/1000
    if p_bid>=(L2[i]):
        L.append(L2[i])
    else:
        L.append(0)

In [373]:
len(L)

299749

In [374]:
new_bid = pd.DataFrame(L,columns=['pre_bid'])

In [375]:
frames=[valid,new_bid]
new_valid=pd.concat(frames,axis=1)

In [376]:
new_valid=new_valid[new_valid.pre_bid!=0]


In [377]:
ctr_cons= sum(new_valid.click)/len(new_valid)
click_cons = sum(new_valid.click)
cost=sum(new_valid["payprice"])/1000
avg_CPM = sum(new_valid.payprice)/1000/(len(new_valid.logtype)/1000)
eCPC = sum(new_valid.payprice)/1000/sum(new_valid.click)
print(ctr_cons)
print(click_cons)
print(cost)
print(avg_CPM)
print(eCPC)

0.000861093881125
118
6111.752
44.5999343233
51.7945084746


In [378]:
cumsum_cost = np.cumsum(new_valid.payprice)
new_valid["cumsum_cost"] = cumsum_cost
new_valid1 = new_valid.loc[(cumsum_cost < 3125000)]
new_valid2 = new_valid.loc[(cumsum_cost < 6250000)]



In [379]:
ctr_cons= sum(new_valid1.click)/len(new_valid1)
click_cons = sum(new_valid1.click)
cost=sum(new_valid1["payprice"])/1000
avg_CPM = sum(new_valid1.payprice)/1000/(len(new_valid1.logtype)/1000)
eCPC = sum(new_valid1.payprice)/1000/sum(new_valid1.click)
print(ctr_cons)
print(click_cons)
print(cost)
print(avg_CPM)
print(eCPC)

0.000967241796225
68
3124.997
44.4504075217
45.9558382353


In [380]:
ctr_cons= sum(new_valid2.click)/len(new_valid2)
click_cons = sum(new_valid2.click)
cost=sum(new_valid2["payprice"])/1000
avg_CPM = sum(new_valid2.payprice)/1000/(len(new_valid2.logtype)/1000)
eCPC = sum(new_valid2.payprice)/1000/sum(new_valid2.click)
print(ctr_cons)
print(click_cons)
print(cost)
print(avg_CPM)
print(eCPC)

0.000861093881125
118
6111.752
44.5999343233
51.7945084746
